# Note on solution depencence of polynomial degree

This small study war born out ouf a problem when trying to develop multigrid solvers for steady-state XDG Stokes problems.
For the example used as a benchmark, the p-multigrid approach does not seem to work.
The main idea here is to use the solution with a low polynomial degree (e.g. degree 1, $k = 1$)
as an initial guess for a higher polynomial degree (most simple, $k = 2$). 
Then, the multigrid approach iterates back and forth between the coarse and the fine solution.
To obtain a "working" multigrid method, i.e. a fast convergence of the method (in a handful of iterations) 
it is required that in the first iteration, the $k = 1$-solution is already a good approximation to the $k = 2$ solution.

This might not always be the case and prevents the multigrid method from working.
The example in below is such a case.

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup

In [ ]:
using BoSSS.Solution.XNSECommon;

Grid resolutions and polynomial degrees:

In [ ]:
int[] GridReS = new int[] { 8, 16, 32, 64, 128, 256 };
int[] DegreeS = new int[] { 1, 2, 3, 4 };

Note that Viscosity in both fluid phases to 1 ($\mu = 1$), to illustrate that the effect in question 
(strong dependence of the solution on the DG polynomial degree) is not just a result of 
large parameter jumps.
Density is also set to 1; However, in steady-state Stokes simulations, density has no effect, since the momentum equation reduces to $\nabla p = \mu \Delta \vec{u}$ and no density is present in the equation.

In [ ]:
XNSE_Control GetControl(int Res, int k) {
    var C = new XNSE_Control();

    C.savetodb = false;
    C.SetDGdegree(k);

    // Air - Water: 
    //C.PhysicalParameters.rho_A = 1e3;
    //C.PhysicalParameters.rho_B = 1.2;
    //C.PhysicalParameters.mu_A = 1e-3;
    //C.PhysicalParameters.mu_B = 17.1e-6;
    C.PhysicalParameters.Sigma = 72.75e-3;

    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.mu_A = 1;
    C.PhysicalParameters.rho_B = C.PhysicalParameters.rho_A;
    C.PhysicalParameters.mu_B = C.PhysicalParameters.mu_A;
            
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material = true;

    double Lscale = 0.01;
    double L = 2.0 * Lscale;

    C.GridFunc = delegate () {
        double[] Xnodes = GenericBlas.Linspace(-(L / 2.0), (L / 2.0), Res + 1);
        double[] Ynodes = GenericBlas.Linspace(-(L / 2.0), (L / 2.0), Res + 1);
        var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);

        grd.DefineEdgeTags(delegate (double[] X) {
            if (Math.Abs(X[1] + (L / 2.0)) <= 1.0e-8)
                return "wall";
            if (Math.Abs(X[1] - (L / 2.0)) <= 1.0e-8)
                return "wall";
            if (Math.Abs(X[0] + (L / 2.0)) <= 1.0e-8)
                return "wall";
            if (Math.Abs(X[0] - (L / 2.0)) <= 1.0e-8)
                return"wall";
            throw new ArgumentException("unable to detect boundary");
        });

        return grd;
    };

    C.AddBoundaryValue("wall");

    double r = 0.25 * Lscale;
    double asym = 1e-2;
    double a = (1 + asym) * r;
    double b = (1 - asym) * r;

    Func<double[], double> PhiFunc = (X => ((X[0] - 0.0).Pow2() / a.Pow2() + (X[1] - 0.0).Pow2() / b.Pow2()) - 1);          // ellipse                     
    C.InitialValues_Evaluators.Add("Phi", PhiFunc);

    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.ConstrainedDG;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;

    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_Flux;
    C.AdvancedDiscretizationOptions.SurfStressTensor = SurfaceSressTensor.Isotropic;

    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();

    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    return C;
}

In [ ]:
(double vel_Range, double p_Range) MyRun(AppControl ctrl) {
    var solverClass = ctrl.GetSolverType();
    object solver = Activator.CreateInstance(solverClass);

    var app = (BoSSS.Application.XNSE_Solver.XNSE<XNSE_Control>) solver;
    app.Init(ctrl);
    app.RunSolverMode();

    app.Velocity[0].GetExtremalValues(out double u_min, out double u_max);
    app.Velocity[1].GetExtremalValues(out double v_min, out double v_max);
    
    app.Pressure.GetExtremalValues(out double p_min, out double p_max);
    double vel_Range = Math.Max(Math.Abs(u_max - u_min), Math.Abs(v_max - v_min));
    double p_Range = Math.Abs(p_max - p_min);
    
    
    Console.WriteLine($"  vel-range: {vel_Range:e4} \tp-range: {p_Range:e4}");
    
    app.Dispose();
    
    return (vel_Range, p_Range);
}

## Execution of runs

Note that, to limit computational costs, for higher polynomial degrees, high resolutions are skipped.

In [ ]:
double[][] vel_Ranges = new double[DegreeS.Length][];
double[][] p_Range = new double[DegreeS.Length][];

In [ ]:
for(int i = 0; i < DegreeS.Length; i++) {
    int k = DegreeS[i];
    
    vel_Ranges[i] = new double[GridReS.Length];
    p_Range[i] = new double[GridReS.Length];
    
    int Count = 0;
    for(int j = 0; j < GridReS.Length; j++) {
        int res = GridReS[j];
        // skip some expensive runs
        if(k >= 2 && res > 64)
            continue;
        if(k >= 3 && res > 32)
            continue;
        Count++;
        var C = GetControl(res, k);
        (vel_Ranges[i][j], p_Range[i][j]) = MyRun(C);
    }
    
    Array.Resize(ref vel_Ranges[i], Count);
    Array.Resize(ref p_Range[i], Count);
}

## Results and Discussion

### Velocity range

In [ ]:
Plot(GridReS.Select(res => Math.Log10(res)).Take(vel_Ranges[0].Length), vel_Ranges[0], "k = 1", "-xr",
     GridReS.Select(res => Math.Log10(res)).Take(vel_Ranges[1].Length), vel_Ranges[1], "k = 2", "-ob",
     GridReS.Select(res => Math.Log10(res)).Take(vel_Ranges[2].Length), vel_Ranges[2], "k = 3", "-+c",
     GridReS.Select(res => Math.Log10(res)).Take(vel_Ranges[3].Length), vel_Ranges[3], "k = 4", "-*m")

One observes a huge difference between different polynomial degrees for coarse meshes, 
especially for k = 1.
This indicates that a p-multigrid (resp. two-grid) method, 
which uses e.g. k = 2 as a fine resolution and k = 1 as a coarse solution would probably never work.

### Pressure range

In [ ]:
Plot(GridReS.Select(res => Math.Log10(res)).Take(p_Range[0].Length), p_Range[0], "k = 1", "-xr",
     GridReS.Select(res => Math.Log10(res)).Take(p_Range[1].Length), p_Range[1], "k = 2", "-ob",
     GridReS.Select(res => Math.Log10(res)).Take(p_Range[2].Length), p_Range[2], "k = 3", "-+c",
     GridReS.Select(res => Math.Log10(res)).Take(p_Range[3].Length), p_Range[3], "k = 4", "-*m")

Surprisingly, one observes that for the pressure the situation is not nearly as severe as for the velocity. 
Even for low resolutions, the pressure range is in a very close range.